# Tutoriel d'usage de l'API

In [1]:
import urllib.parse

domain = "51.91.251.0"
port = 3000
host = f"http://{domain}:{port}"
path = lambda x: urllib.parse.urljoin(host, x)

In [2]:
import numpy as np 

## Creating an avatar

The admin of the competition can create users. A user can then create as many avatars as they wish.

In [152]:
import requests

user_id = '12e1e6fc-456b-4738-ba59-b90c3386f583'
name = 'ema-estanian1-avatar'
r = requests.post(path(f'avatars/{user_id}/{name}'))
r

<Response [200]>

In [153]:
# name = 'max-second-avatar'
# r = requests.post(path(f"avatars/{user_id}/{name}"))
# r

In [154]:
r = requests.get(path(f"avatars/{user_id}"))
for avatar in r.json():
    print(avatar['id'], avatar['name'])

6217 ema-first-avatar
34250 ema-austrian1-avatar
34256 ema-belgian1-avatar
34257 ema-bulgarian1-avatar
34258 ema-croatian1-avatar
34259 ema-cypriot1-avatar
34260 ema-czech1-avatar
34261 ema-danish1-avatar
34262 ema-dutch1-avatar
34263 ema-estanian1-avatar


## Making pricing requests

In [64]:
language_list=['austrian', 'belgian', 'bulgarian', 'croatian', 'cypriot', 'czech', 'danish', 'dutch', 'estonian', 'finnish', 'french', 'german', 'greek', 'hungarian', 'irish', 'italian', 'latvian', 'lithuanian', 'luxembourgish', 'maltese', 'polish', 'portuguese', 'romanian', 'slovakian', 'slovene', 'spanish' ,'swedish']

city_list=['amsterdam', 'copenhagen', 'madrid', 'paris', 'rome', 'sofia', 'valletta', 'vienna','vilnius']

date_list=list(np.arange(1,45,1))

In [155]:
## A modifier logiquement selon dates souhaitées

date_list_soon=[3,5,7,8]

date_list_middle=[7,9,11,13,14]

date_list_away=[35,37,40,41]



In [163]:
language_test='estonian'
avatar_test='ema-estanian1-avatar'


In [164]:
print(avatar_test, language_test,date_list_away)

ema-estanian1-avatar estonian [35, 37, 40, 41]


In [165]:
requests_list=[]

for date in reversed(date_list_away) : 
    for city in city_list : 
        params = {
            "avatar_name": avatar_test,
            "language": language_test,
            "city": city,
            "date": date,
            "mobile": np.random.randint(0,2),
        }
        r = requests.get(path(f"pricing/{user_id}"), params=params)
        r.json()
        
        requests_list.append(r)
       

## Build a training set

In [166]:
import pandas as pd

pricing_requests = []

for r in requests_list:
    pricing_requests.append(
        pd.DataFrame(r.json()['prices']).assign(**r.json()['request'])
    )

pricing_requests = pd.concat(pricing_requests)
pricing_requests.head()

,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,960,234,233,amsterdam,41,estonian,1,34263
1,788,99,116,amsterdam,41,estonian,1,34263
2,117,71,46,amsterdam,41,estonian,1,34263
3,918,223,226,amsterdam,41,estonian,1,34263
4,971,132,116,amsterdam,41,estonian,1,34263


In [167]:
len(pricing_requests)

3794

In [144]:
# print(pricing_requests[ pricing_requests['mobile'] == 0])

In [145]:
data=pd.read_csv('./Datatrain/test_requete.csv')

In [146]:
data=data.drop(['Unnamed: 0'],axis=1)

In [147]:
pricing_requests=pd.concat([data,pricing_requests])

In [148]:
pricing_requests

,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,556,175,60,copenhagen,30,hungarian,0,6217
1,664,284,111,copenhagen,30,hungarian,0,6217
2,96,124,37,copenhagen,30,hungarian,0,6217
3,865,277,107,copenhagen,30,hungarian,0,6217
4,58,203,65,copenhagen,30,hungarian,0,6217
...,...,...,...,...,...,...,...,...
81,362,85,62,vilnius,35,dutch,0,34262
82,687,112,78,vilnius,35,dutch,0,34262
83,654,141,109,vilnius,35,dutch,0,34262
84,18,55,35,vilnius,35,dutch,0,34262


In [149]:
pricing_requests.to_csv('./Datatrain/test_requete.csv')

In [150]:
hotels = pd.read_csv('features_hotels.csv', index_col=['hotel_id', 'city'])
pricing_requests = pricing_requests.join(hotels, on=['hotel_id', 'city'])
pricing_requests.head()

,hotel_id,price,stock,city,date,language,mobile,avatar_id,group,brand,parking,pool,children_policy
0,556,175,60,copenhagen,30,hungarian,0,6217,Boss Western,J.Halliday Inn,1,0,0
1,664,284,111,copenhagen,30,hungarian,0,6217,Accar Hotels,Safitel,0,1,0
2,96,124,37,copenhagen,30,hungarian,0,6217,Boss Western,Boss Western,0,0,0
3,865,277,107,copenhagen,30,hungarian,0,6217,Chillton Worldwide,Chill Garden Inn,1,1,0
4,58,203,65,copenhagen,30,hungarian,0,6217,Yin Yang,Royal Lotus,1,0,2


In [151]:
pricing_requests.to_csv('./Datatrain/test_requete_features.csv')

In [132]:
# sample_submission = pd.read_csv('../input/defi-ia-2023/sample_submission.csv')
# sample_submission.head()

In [12]:
# sample_submission.to_csv('submission.csv', index=False)